In [82]:
import pandas as pd
import open3d as o3d
import numpy as np
import math
import csv
import copy
import os

In [83]:
LiDAR_data = '../data/lidar/300/2022-12-10-16-53-36_Velodyne-VLP-16-Data.csv'

os.path.exists(LiDAR_data)

True

In [84]:
with open(LiDAR_data, newline='') as f:
    rows = list(csv.reader(f, delimiter=',', quotechar='"'))
    pts_lidar = np.zeros(shape=(len(rows) - 1, 3))
    headers = rows.pop(0)
    for index, row in enumerate(rows[1:]):
        # print(index, row[0])
        pts_lidar[index] = row[0:3]
print(pts_lidar.shape)

(17556, 3)


In [85]:
def appendSpherical_np(xyz):
    ptsnew = np.hstack((xyz, np.zeros(xyz.shape)))
    xy = xyz[:,0]**2 + xyz[:,1]**2
    ptsnew[:,3] = np.sqrt(xy + xyz[:,2]**2)
    ptsnew[:,4] = np.arctan2(np.sqrt(xy), xyz[:,2]) # for elevation angle defined from Z-axis down
    #ptsnew[:,4] = np.arctan2(xyz[:,2], np.sqrt(xy)) # for elevation angle defined from XY-plane up
    ptsnew[:,5] = np.arctan2(xyz[:,1], xyz[:,0])
    return ptsnew

In [86]:
def sph2cart(r, theta, phi):
    return [
         r * math.sin(theta) * math.cos(phi),
         r * math.sin(theta) * math.sin(phi),
         r * math.cos(theta)
    ]

In [87]:
pts_lidar_sph = appendSpherical_np(pts_lidar)
vert_angles = pts_lidar_sph[:,4]
hor_angles = pts_lidar_sph[:,5]
dist = pts_lidar_sph[:,3]

In [88]:
for i in range(len(dist)):
    pts_lidar_back = sph2cart(dist[i], vert_angles[i], hor_angles[i])
print(pts_lidar-pts_lidar_back)

TypeError: only size-1 arrays can be converted to Python scalars